# Extração dos dados da API do Olho vivo

Nesse notebook são extraídos os dados sobre os ônibus do município

Os dados são da API do Olho vivo: https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-referencia/

In [1]:
import pandas as pd
import time
import random
from requests import Session
from typing import Optional
from utils.save_csv import save_csv

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

espera = 30
for i in range(2):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.
Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '12:25',
  'l': [{'c': '4014-10',
    'cl': 2187,
    'sl': 1,
    'lt0': 'TERM. VL. CARRÃO',
    'lt1': 'JD. VL. CARRÃO',
    'qv': 4,
    'vs': [{'p': 48729,
      'a': True,
      'ta': '2025-08-28T15:25:30Z',
      'py': -23.611476500000002,
      'px': -46.4564085,
      'sv': None,
      'is': None},
     {'p': 48062,
      'a': True,
      'ta': '2025-08-28T15:24:59Z',
      'py': -23.612996,
      'px': -46.456739999999996,
      'sv': None,
      'is': None},
     {'p': 48274,
      'a': True,
      'ta': '2025-08-28T15:25:36Z',
      'py': -23.5783385,
      'px': -46.5030845,
      'sv': None,
      'is': None},
     {'p': 48866,
      'a': True,
      'ta': '2025-08-28T15:25:23Z',
      'py': -23.59631125,
      'px': -46.47557625,
      'sv': None,
      'is': None}]},
   {'c': '648P-10',
    'cl': 32834,
    'sl': 2,
    'lt0': 'TERM. PINHEIROS',
    'lt1': 'TERM. CAPELINHA',
    'qv': 5,
    'vs': [{'p': 71429,
      'a': True,
      'ta': '2025-08-28T15:25:42Z',

In [6]:
rows = []

for l in posicoes[0]['l']:

    for veiculo in l['vs']:

        row = {
            'linha': l['c'],
            'id_onibus': veiculo['p'],
            'timestamp': veiculo['ta'],
            'lat': veiculo['py'],
            'lon': veiculo['px'] 
        }

        rows.append(row)

df_posicoes = pd.DataFrame(rows)

Aleatorização dos ônibus elétricos

In [7]:
df_posicoes['is_eletrico'] = [random.choice([True, False]) for i in range(len(df_posicoes))]

Exibir e salvar

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon,is_eletrico
0,4014-10,48729,2025-08-28T15:25:30Z,-23.611477,-46.456409,True
1,4014-10,48062,2025-08-28T15:24:59Z,-23.612996,-46.456740,True
2,4014-10,48274,2025-08-28T15:25:36Z,-23.578339,-46.503084,True
3,4014-10,48866,2025-08-28T15:25:23Z,-23.596311,-46.475576,True
4,648P-10,71429,2025-08-28T15:25:42Z,-23.566113,-46.701726,False
...,...,...,...,...,...,...
10016,8300-10,11772,2025-08-28T15:25:31Z,-23.513772,-46.702571,True
10017,546J-31,68593,2025-08-28T15:25:06Z,-23.674820,-46.661744,False
10018,546J-31,68184,2025-08-28T15:25:17Z,-23.686341,-46.656746,True
10019,4286-10,51003,2025-08-28T15:24:59Z,-23.598073,-46.539898,False


In [9]:
save_csv(df_posicoes, 'df_posicoes.csv')

Base salva em data\df_posicoes.csv
